In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/archive.zip -d /content/data/


Streaming output truncated to the last 5000 lines.
  inflating: /content/data/dataset/Train/rottencapsicum/WhatsApp Image 2020-11-07 at 11.43.17 PM.jpg_0_5372.jpg  
  inflating: /content/data/dataset/Train/rottencapsicum/WhatsApp Image 2020-11-07 at 11.43.17 PM.jpg_0_54.jpg  
  inflating: /content/data/dataset/Train/rottencapsicum/WhatsApp Image 2020-11-07 at 11.43.17 PM.jpg_0_6186.jpg  
  inflating: /content/data/dataset/Train/rottencapsicum/WhatsApp Image 2020-11-07 at 11.43.17 PM.jpg_0_6285.jpg  
  inflating: /content/data/dataset/Train/rottencapsicum/WhatsApp Image 2020-11-07 at 11.43.17 PM.jpg_0_6551.jpg  
  inflating: /content/data/dataset/Train/rottencapsicum/WhatsApp Image 2020-11-07 at 11.43.17 PM.jpg_0_6565.jpg  
  inflating: /content/data/dataset/Train/rottencapsicum/WhatsApp Image 2020-11-07 at 11.43.17 PM.jpg_0_6985.jpg  
  inflating: /content/data/dataset/Train/rottencapsicum/WhatsApp Image 2020-11-07 at 11.43.17 PM.jpg_0_7173.jpg  
  inflating: /content/data/dataset/Trai

In [4]:
print(f"Listing contents of {TRAIN_DIR}:")
!ls -F {TRAIN_DIR}
print(f"\nListing contents of {VALIDATION_DIR}:")
!ls -F {VALIDATION_DIR}
print(f"\nListing contents of {TEST_DIR}:")
!ls -F {TEST_DIR}

Listing contents of /content/data/dataset/Train/:
freshapples/	   freshokra/	  rottenbanana/       rottenoranges/
freshbanana/	   freshoranges/  rottenbittergroud/  rottenpotato/
freshbittergroud/  freshpotato/   rottencapsicum/     rottentomato/
freshcapsicum/	   freshtomato/   rottencucumber/
freshcucumber/	   rottenapples/  rottenokra/

Listing contents of /content/data/dataset/Validation/:
ls: cannot access '/content/data/dataset/Validation/': No such file or directory

Listing contents of /content/data/dataset/Test/:
freshapples/	freshokra/     freshtamto/    rottencucumber/  rottenpatato/
freshbanana/	freshoranges/  rottenapples/  rottenokra/      rottentamto/
freshcucumber/	freshpatato/   rottenbanana/  rottenoranges/


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 80/20 split for training/validation
)

validation_test_datagen = ImageDataGenerator(rescale=1./255)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

# Validation data generator
validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Test data generator
test_generator = validation_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

print("Image data generators created successfully.")

Found 18901 images belonging to 18 classes.
Found 4718 images belonging to 18 classes.
Found 6738 images belonging to 14 classes.
Image data generators created successfully.


In [7]:
print("Train Generator Classes:", train_generator.class_indices)
print("Validation Generator Classes:", validation_generator.class_indices)
print("Test Generator Classes:", test_generator.class_indices)


Train Generator Classes: {'freshapples': 0, 'freshbanana': 1, 'freshbittergroud': 2, 'freshcapsicum': 3, 'freshcucumber': 4, 'freshokra': 5, 'freshoranges': 6, 'freshpotato': 7, 'freshtomato': 8, 'rottenapples': 9, 'rottenbanana': 10, 'rottenbittergroud': 11, 'rottencapsicum': 12, 'rottencucumber': 13, 'rottenokra': 14, 'rottenoranges': 15, 'rottenpotato': 16, 'rottentomato': 17}
Validation Generator Classes: {'freshapples': 0, 'freshbanana': 1, 'freshbittergroud': 2, 'freshcapsicum': 3, 'freshcucumber': 4, 'freshokra': 5, 'freshoranges': 6, 'freshpotato': 7, 'freshtomato': 8, 'rottenapples': 9, 'rottenbanana': 10, 'rottenbittergroud': 11, 'rottencapsicum': 12, 'rottencucumber': 13, 'rottenokra': 14, 'rottenoranges': 15, 'rottenpotato': 16, 'rottentomato': 17}
Test Generator Classes: {'freshapples': 0, 'freshbanana': 1, 'freshcucumber': 2, 'freshokra': 3, 'freshoranges': 4, 'freshpatato': 5, 'freshtamto': 6, 'rottenapples': 7, 'rottenbanana': 8, 'rottencucumber': 9, 'rottenokra': 10, '

In [8]:
import os

# Define a dictionary to correct misspelled class names
class_name_mapping = {
    'freshpatato': 'freshpotato',
    'freshtamto': 'freshtomato',
    'rottenpatato': 'rottenpotato',
    'rottentamto': 'rottentomato'
}

# Correct class names in the TEST_DIR
print(f"Correcting class names in {TEST_DIR}...")
for class_dir in os.listdir(TEST_DIR):
    old_path = os.path.join(TEST_DIR, class_dir)
    if os.path.isdir(old_path) and class_dir in class_name_mapping:
        new_class_name = class_name_mapping[class_dir]
        new_path = os.path.join(TEST_DIR, new_class_name)
        os.rename(old_path, new_path)
        print(f"Renamed '{class_dir}' to '{new_class_name}' in test directory.")
print("Class name correction complete.")

# Identify the final set of class names present in the TEST_DIR after renaming
common_classes = sorted([d for d in os.listdir(TEST_DIR) if os.path.isdir(os.path.join(TEST_DIR, d))])
print(f"Common classes identified: {common_classes}")

# Re-initialize ImageDataGenerator instances
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 80/20 split for training/validation
)

validation_test_datagen = ImageDataGenerator(rescale=1./255)

# Recreate Training data generator with filtered common classes
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    classes=common_classes # Filter to common classes
)

# Recreate Validation data generator with filtered common classes
validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    classes=common_classes # Filter to common classes
)

# Recreate Test data generator with filtered common classes
test_generator = validation_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    classes=common_classes # Filter to common classes
)

print("Image data generators recreated successfully with consistent classes.")

print("\nTrain Generator Classes:", train_generator.class_indices)
print("Validation Generator Classes:", validation_generator.class_indices)
print("Test Generator Classes:", test_generator.class_indices)


Correcting class names in /content/data/dataset/Test/...
Renamed 'freshtamto' to 'freshtomato' in test directory.
Renamed 'freshpatato' to 'freshpotato' in test directory.
Renamed 'rottentamto' to 'rottentomato' in test directory.
Renamed 'rottenpatato' to 'rottenpotato' in test directory.
Class name correction complete.
Common classes identified: ['freshapples', 'freshbanana', 'freshcucumber', 'freshokra', 'freshoranges', 'freshpotato', 'freshtomato', 'rottenapples', 'rottenbanana', 'rottencucumber', 'rottenokra', 'rottenoranges', 'rottenpotato', 'rottentomato']
Found 16840 images belonging to 14 classes.
Found 4204 images belonging to 14 classes.
Found 6738 images belonging to 14 classes.
Image data generators recreated successfully with consistent classes.

Train Generator Classes: {'freshapples': 0, 'freshbanana': 1, 'freshcucumber': 2, 'freshokra': 3, 'freshoranges': 4, 'freshpotato': 5, 'freshtomato': 6, 'rottenapples': 7, 'rottenbanana': 8, 'rottencucumber': 9, 'rottenokra': 10,

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Initialize the CNN model
model = Sequential()

# Convolutional Layer 1
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional Layer 2
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional Layer 3
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening
model.add(Flatten())

# Fully Connected Layers
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5)) # Added Dropout for regularization
model.add(Dense(train_generator.num_classes, activation='softmax')) # Output layer

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     9,470,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 14)             │         3,598 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,567,054 (36.50 MB)

 Trainable params: 9,567,054 (36.50 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

# Initialize the CNN model
model = Sequential([
    Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
    # Convolutional Layer 1
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Convolutional Layer 2
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Convolutional Layer 3
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Flattening
    Flatten(),

    # Fully Connected Layers
    Dense(256, activation='relu'),
    Dropout(0.5), # Added Dropout for regularization
    Dense(train_generator.num_classes, activation='softmax') # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │     9,470,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 14)             │         3,598 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,567,054 (36.50 MB)

 Trainable params: 9,567,054 (36.50 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
EPOCHS = 10

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

print("Model training complete.")

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
526/526 ━━━━━━━━━━━━━━━━━━━━ 1107s 2s/step - accuracy: 0.5496 - loss: 1.3516 - val_accuracy: 0.7125 - val_loss: 0.7473
Epoch 2/10
  1/526 ━━━━━━━━━━━━━━━━━━━━ 12:37 1s/step - accuracy: 0.6562 - loss: 0.7989

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


526/526 ━━━━━━━━━━━━━━━━━━━━ 98s 184ms/step - accuracy: 0.6562 - loss: 0.7989 - val_accuracy: 0.7135 - val_loss: 0.7481
Epoch 3/10
526/526 ━━━━━━━━━━━━━━━━━━━━ 1164s 2s/step - accuracy: 0.8060 - loss: 0.5333 - val_accuracy: 0.7314 - val_loss: 0.7890
Epoch 4/10
526/526 ━━━━━━━━━━━━━━━━━━━━ 98s 184ms/step - accuracy: 0.8438 - loss: 0.4790 - val_accuracy: 0.7312 - val_loss: 0.7644
Epoch 5/10
526/526 ━━━━━━━━━━━━━━━━━━━━ 1136s 2s/step - accuracy: 0.8472 - loss: 0.4076 - val_accuracy: 0.7345 - val_loss: 0.7717
Epoch 6/10
  1/526 ━━━━━━━━━━━━━━━━━━━━ 13:05 1s/step - accuracy: 0.8438 - loss: 0.3351

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


526/526 ━━━━━━━━━━━━━━━━━━━━ 100s 188ms/step - accuracy: 0.8438 - loss: 0.3351 - val_accuracy: 0.7369 - val_loss: 0.7313
Epoch 7/10
431/526 ━━━━━━━━━━━━━━━━━━━━ 2:57 2s/step - accuracy: 0.8619 - loss: 0.3652

In [12]:
test_loss, test_accuracy = model.evaluate(
    test_generator,
    steps=test_generator.samples // BATCH_SIZE
)

print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


210/210 ━━━━━━━━━━━━━━━━━━━━ 124s 587ms/step - accuracy: 0.9104 - loss: 0.2312

Test Loss: 0.2997
Test Accuracy: 0.8821


In [13]:
test_loss, test_accuracy = model.evaluate(
    test_generator,
    steps=test_generator.samples // BATCH_SIZE
)

print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

210/210 ━━━━━━━━━━━━━━━━━━━━ 118s 560ms/step - accuracy: 0.9104 - loss: 0.2312

Test Loss: 0.2997
Test Accuracy: 0.8821


In [15]:
model.save('food_freshness_model.h5')
print("Model saved successfully as 'food_freshness_model.h5'.")

Model saved successfully as 'food_freshness_model.h5'.


In [16]:
model.save('food_freshness_model.keras')
print("Model saved successfully as 'food_freshness_model.keras'.")

Model saved successfully as 'food_freshness_model.keras'.


In [17]:
import os
import numpy as np
from flask import Flask, render_template, request, redirect, url_for
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

print("Required libraries imported.")

Required libraries imported.


In [18]:
app = Flask(__name__)
UPLOAD_FOLDER = 'uploads'
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

# Load the trained model
model = load_model('food_freshness_model.keras')

# Define class names from the common_classes list
class_names = common_classes

print("Flask app initialized, uploads directory created, model loaded, and class names defined.")

Flask app initialized, uploads directory created, model loaded, and class names defined.


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [19]:
from werkzeug.utils import secure_filename

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif'}

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            filepath = os.path.join(UPLOAD_FOLDER, filename)
            file.save(filepath)

            # Preprocess the image
            img = image.load_img(filepath, target_size=(IMG_WIDTH, IMG_HEIGHT))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0) / 255.0

            # Make prediction
            predictions = model.predict(img_array)
            predicted_class_index = np.argmax(predictions[0])
            predicted_class_name = class_names[predicted_class_index]

            return render_template('result.html', filename=filename, prediction=predicted_class_name)
    return render_template('index.html')

print("Flask app route for file upload and prediction defined.")

Flask app route for file upload and prediction defined.


In [22]:
os.makedirs('templates', exist_ok=True)
print("Created 'templates' directory.")

Created 'templates' directory.


In [23]:
%%writefile templates/index.html
<!doctype html>
<title>Upload new File</title>
<h1>Upload new File</h1>
<form method=post enctype=multipart/form-data>
  <input type=file name=file>
  <input type=submit value=Upload>
</form>

Writing templates/index.html


In [24]:
%%writefile templates/result.html
<!doctype html>
<title>Prediction Result</title>
<h1>Prediction Result</h1>
<p>Uploaded image: <img src="{{ url_for('static', filename='uploads/' + filename) }}" width="200"></p>
<p>Prediction: <strong>{{ prediction }}</strong></p>
<p><a href="/">Upload another file</a></p>

Writing templates/result.html


In [25]:
import os
import numpy as np
from fastapi import FastAPI, UploadFile, File
from PIL import Image
import io
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

print("Required libraries imported for FastAPI application.")

Required libraries imported for FastAPI application.


In [26]:
app = FastAPI()

# Load the trained model
model = load_model('food_freshness_model.keras')

# Define class names from the common_classes list
class_names = common_classes

@app.get("/")
async def read_root():
    return {"message": "Welcome to the Food Freshness Classification API"}

@app.post("/predict")
async def predict_image(file: UploadFile = File(...)):
    # Read the image file
    contents = await file.read()
    img = Image.open(io.BytesIO(contents))

    # Preprocess the image
    img = img.resize((IMG_WIDTH, IMG_HEIGHT))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    # Make prediction
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class_name = class_names[predicted_class_index]

    return {"filename": file.filename, "prediction": predicted_class_name}

print("FastAPI app initialized, model loaded, class names defined, and endpoints created.")

FastAPI app initialized, model loaded, class names defined, and endpoints created.


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [27]:
import uvicorn

# Run the FastAPI application
# The port can be changed if needed
# This will run on http://127.0.0.1:8000 by default,
# but if running in Colab, you'll need a tool like ngrok to expose it.
print("To run the FastAPI app locally, execute the following command in your terminal:")
print("uvicorn main:app --host 0.0.0.0 --port 8000")
print("Or, to run it within this Colab environment (requires ngrok for external access):")
print("!pip install pyngrok")
print("from pyngrok import ngrok\npublic_url = ngrok.connect(8000)\nprint(f\"Public URL: {public_url}\")\nuvicorn.run(app, host=\"0.0.0.0\", port=8000)")

print("FastAPI application setup complete. You can now run the server using the instructions above.")

To run the FastAPI app locally, execute the following command in your terminal:
uvicorn main:app --host 0.0.0.0 --port 8000
Or, to run it within this Colab environment (requires ngrok for external access):
!pip install pyngrok
from pyngrok import ngrok
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")
uvicorn.run(app, host="0.0.0.0", port=8000)
FastAPI application setup complete. You can now run the server using the instructions above.


In [29]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.2 MB/s eta 0:00:00


In [30]:
import streamlit as st
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image
import io

print("Required libraries imported for Streamlit application.")

Required libraries imported for Streamlit application.


In [36]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from flask import Flask, render_template, request, redirect, url_for
from werkzeug.utils import secure_filename
from PIL import Image
import io
import gradio as gr

print("All necessary libraries imported.")

All necessary libraries imported.


In [37]:
# Define constants for image dimensions and batch size
IMG_HEIGHT = 150
IMG_WIDTH = 150
BATCH_SIZE = 32

# Specify the paths to the dataset directories
TRAIN_DIR = '/content/data/dataset/Train/'
TEST_DIR = '/content/data/dataset/Test/'

# Create the 'uploads' and 'templates' directories for Flask if they don't exist
UPLOAD_FOLDER = 'uploads'
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

if not os.path.exists('templates'):
    os.makedirs('templates')

print("Constants defined and directories ensured.")

Constants defined and directories ensured.


In [38]:
# Define a dictionary to correct misspelled class names
class_name_mapping = {
    'freshpatato': 'freshpotato',
    'freshtamto': 'freshtomato',
    'rottenpatato': 'rottenpotato',
    'rottentamto': 'rottentomato'
}

# Correct class names in the TEST_DIR
print(f"Correcting class names in {TEST_DIR}...")
for class_dir in os.listdir(TEST_DIR):
    old_path = os.path.join(TEST_DIR, class_dir)
    if os.path.isdir(old_path) and class_dir in class_name_mapping:
        new_class_name = class_name_mapping[class_dir]
        new_path = os.path.join(TEST_DIR, new_class_name)
        os.rename(old_path, new_path)
        print(f"Renamed '{class_dir}' to '{new_class_name}' in test directory.")
print("Class name correction complete.")

# Identify the final set of class names present in the TEST_DIR after renaming
common_classes = sorted([d for d in os.listdir(TEST_DIR) if os.path.isdir(os.path.join(TEST_DIR, d))])
print(f"Common classes identified: {common_classes}")

Correcting class names in /content/data/dataset/Test/...
Class name correction complete.
Common classes identified: ['freshapples', 'freshbanana', 'freshcucumber', 'freshokra', 'freshoranges', 'freshpotato', 'freshtomato', 'rottenapples', 'rottenbanana', 'rottencucumber', 'rottenokra', 'rottenoranges', 'rottenpotato', 'rottentomato']


In [39]:
# Data augmentation and preprocessing for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 80/20 split for training/validation
)

# Data rescaling for validation and test sets
validation_test_datagen = ImageDataGenerator(rescale=1./255)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    classes=common_classes # Filter to common classes
)

# Validation data generator (from training set)
validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    classes=common_classes # Filter to common classes
)

# Test data generator
test_generator = validation_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    classes=common_classes # Filter to common classes
)

print("Image data generators created successfully with consistent classes.")

Found 16840 images belonging to 14 classes.
Found 4204 images belonging to 14 classes.
Found 6738 images belonging to 14 classes.
Image data generators created successfully with consistent classes.


In [40]:
# Initialize the CNN model
model = Sequential([
    Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
    # Convolutional Layer 1
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Convolutional Layer 2
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Convolutional Layer 3
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Flattening
    Flatten(),

    # Fully Connected Layers
    Dense(256, activation='relu'),
    Dropout(0.5), # Added Dropout for regularization
    Dense(train_generator.num_classes, activation='softmax') # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()
print("CNN model built and compiled.")

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │     9,470,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 14)             │         3,598 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,567,054 (36.50 MB)

 Trainable params: 9,567,054 (36.50 MB)

 Non-trainable params: 0 (0.00 B)

CNN model built and compiled.


In [42]:
EPOCHS = 10

print(f"Training model for {EPOCHS} epochs...")
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

print("Model training complete.")

Training model for 10 epochs...
Epoch 1/10
526/526 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7761 - loss: 0.6247

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


526/526 ━━━━━━━━━━━━━━━━━━━━ 1126s 2s/step - accuracy: 0.7761 - loss: 0.6246 - val_accuracy: 0.7216 - val_loss: 0.7062
Epoch 2/10
  1/526 ━━━━━━━━━━━━━━━━━━━━ 20:03 2s/step - accuracy: 0.8438 - loss: 0.4490

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


526/526 ━━━━━━━━━━━━━━━━━━━━ 143s 267ms/step - accuracy: 0.8438 - loss: 0.4490 - val_accuracy: 0.7366 - val_loss: 0.6857
Epoch 3/10
526/526 ━━━━━━━━━━━━━━━━━━━━ 1130s 2s/step - accuracy: 0.8266 - loss: 0.4614 - val_accuracy: 0.7662 - val_loss: 0.6032
Epoch 4/10
526/526 ━━━━━━━━━━━━━━━━━━━━ 102s 190ms/step - accuracy: 0.7500 - loss: 0.5073 - val_accuracy: 0.7708 - val_loss: 0.6080
Epoch 5/10
526/526 ━━━━━━━━━━━━━━━━━━━━ 1172s 2s/step - accuracy: 0.8581 - loss: 0.3766 - val_accuracy: 0.7595 - val_loss: 0.6966
Epoch 6/10
526/526 ━━━━━━━━━━━━━━━━━━━━ 101s 187ms/step - accuracy: 0.9375 - loss: 0.2650 - val_accuracy: 0.7538 - val_loss: 0.7417
Epoch 7/10
526/526 ━━━━━━━━━━━━━━━━━━━━ 1103s 2s/step - accuracy: 0.8749 - loss: 0.3250 - val_accuracy: 0.7643 - val_loss: 0.7244
Epoch 8/10
526/526 ━━━━━━━━━━━━━━━━━━━━ 142s 267ms/step - accuracy: 0.8125 - loss: 0.3901 - val_accuracy: 0.7834 - val_loss: 0.6915
Epoch 9/10
526/526 ━━━━━━━━━━━━━━━━━━━━ 1158s 2s/step - accuracy: 0.8863 - loss: 0.3027 - val

In [ ]:
test_loss, test_accuracy = model.evaluate(
    test_generator,
    steps=test_generator.samples // BATCH_SIZE
)

print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
model.save('food_freshness_model.keras')
print("Model saved successfully as 'food_freshness_model.keras'.")

### Flask Web Interface: index.html
```html
<!doctype html>
<title>Upload new File</title>
<h1>Upload new File</h1>
<form method=post enctype=multipart/form-data>
  <input type=file name=file>
  <input type=submit value=Upload>
</form>
```

### Flask Web Interface: result.html
```html
<!doctype html>
<title>Prediction Result</title>
<h1>Prediction Result</h1>
<p>Uploaded image: <img src="{{ url_for('static', filename='uploads/' + filename) }}" width="200"></p>
<p>Prediction: <strong>{{ prediction }}</strong></p>
<p><a href="/">Upload another file</a></p>
```

In [ ]:
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif'}

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(filepath)

            img = tf.keras.utils.load_img(filepath, target_size=(IMG_WIDTH, IMG_HEIGHT))
            img_array = tf.keras.utils.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0) / 255.0

            predictions = model.predict(img_array)
            predicted_class_index = np.argmax(predictions[0])
            predicted_class_name = common_classes[predicted_class_index]

            return render_template('result.html', filename=filename, prediction=predicted_class_name)
    return render_template('index.html')

print("Flask application routes defined.")
print("To run the Flask app, execute the following command in a new terminal in your environment:")
print("FLASK_APP=app.py flask run")
print("If running in Colab, save the app code to a file (e.g., `flask_app.py`) and use ngrok for public access, similar to FastAPI instructions.")

In [ ]:
from fastapi import FastAPI, UploadFile, File
from PIL import Image
import io
import uvicorn

fastapi_app = FastAPI()

@fastapi_app.get("/")
async def read_root():
    return {"message": "Welcome to the Food Freshness Classification API"}

@fastapi_app.post("/predict")
async def predict_image(file: UploadFile = File(...)):
    contents = await file.read()
    img = Image.open(io.BytesIO(contents))

    img = img.resize((IMG_WIDTH, IMG_HEIGHT))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class_name = common_classes[predicted_class_index]

    return {"filename": file.filename, "prediction": predicted_class_name}

print("FastAPI application endpoints defined.")
print("To run the FastAPI app, save the above code to a file (e.g., `fastapi_app.py`) and then execute:")
print("uvicorn fastapi_app:fastapi_app --host 0.0.0.0 --port 8000")
print("Or, for Colab with ngrok:")
print("!pip install pyngrok -q")
print("from pyngrok import ngrok")
print("public_url = ngrok.connect(8000)")
print("print(f\"Public URL: {public_url}\")")
print("uvicorn.run(fastapi_app, host=\"0.0.0.0\", port=8000)")

In [ ]:
%%writefile streamlit_app.py
import streamlit as st
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image
import io

IMG_WIDTH = 150
IMG_HEIGHT = 150
common_classes = ['freshapples', 'freshbanana', 'freshcucumber', 'freshokra', 'freshoranges', 'freshpotato', 'freshtomato', 'rottenapples', 'rottenbanana', 'rottencucumber', 'rottenokra', 'rottenoranges', 'rottenpotato', 'rottentomato']

st.title("Food Freshness Classifier")
st.write("Upload an image to classify its freshness.")

model = load_model('food_freshness_model.keras')

class_names = common_classes

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png", "gif"])

if uploaded_file is not None:
    image_data = uploaded_file.read()
    st.image(image_data, caption='Uploaded Image.', use_column_width=True)
    st.write("")
    st.write("Classifying...")

    img = Image.open(io.BytesIO(image_data))
    img = img.resize((IMG_WIDTH, IMG_HEIGHT))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class_name = class_names[predicted_class_index]

    st.write(f"Prediction: **{predicted_class_name}**")

print("Streamlit application script 'streamlit_app.py' created.")
print("\nTo run the Streamlit app, execute the following commands:")
print("!pip install pyngrok -q")
print("from pyngrok import ngrok")
print("public_url = ngrok.connect(8501)")
print("print(f\"Streamlit App URL: {public_url}\")")
print("!streamlit run streamlit_app.py --server.port 8501 --server.enableCORS false --server.enableXsrfProtection false")

In [ ]:
import gradio as gr

def predict_freshness(img: Image.Image) -> str:
    img = img.resize((IMG_WIDTH, IMG_HEIGHT))
    img_array = np.asarray(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class_name = common_classes[predicted_class_index]

    return predicted_class_name

iface = gr.Interface(
    fn=predict_freshness,
    inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=gr.Label(),
    title="Food Freshness Classifier",
    description="Upload an image of food to classify its freshness (e.g., freshapples, rottenbanana)."
)

print("Launching Gradio interface. This might take a moment...")
iface.launch(share=True)

In [31]:
st.title("Food Freshness Classifier")
st.write("Upload an image to classify its freshness.")

# Load the trained model
model = load_model('food_freshness_model.keras')

# Define class names from the common_classes list (already available in kernel state)
class_names = common_classes

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png", "gif"])

if uploaded_file is not None:
    # Display the uploaded image
    image_data = uploaded_file.read()
    st.image(image_data, caption='Uploaded Image.', use_column_width=True)
    st.write("")
    st.write("Classifying...")

    # Preprocess the image
    img = Image.open(io.BytesIO(image_data))
    img = img.resize((IMG_WIDTH, IMG_HEIGHT))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    # Make prediction
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class_name = class_names[predicted_class_index]

    st.write(f"Prediction: **{predicted_class_name}**")

print("Streamlit application logic defined.")

2025-12-12 14:07:14.584 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-12 14:07:14.747 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-12-12 14:07:14.748 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-12 14:07:14.750 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-12 14:07:14.751 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-12 14:07:14.753 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-12 14:07:14.755 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
/usr/local/lib/python3.12/dist-packages/keras/s

Streamlit application logic defined.


In [32]:
%%writefile app.py
import streamlit as st
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image
import io

# Assuming IMG_WIDTH, IMG_HEIGHT, and common_classes are defined globally in the notebook
# and we are transferring them to the Streamlit app context for clarity/execution.
# In a real app.py, these would be defined directly or loaded.
IMG_WIDTH = 150
IMG_HEIGHT = 150
common_classes = ['freshapples', 'freshbanana', 'freshcucumber', 'freshokra', 'freshoranges', 'freshpotato', 'freshtomato', 'rottenapples', 'rottenbanana', 'rottencucumber', 'rottenokra', 'rottenoranges', 'rottenpotato', 'rottentomato']

st.title("Food Freshness Classifier")
st.write("Upload an image to classify its freshness.")

# Load the trained model
model = load_model('food_freshness_model.keras')

# Define class names from the common_classes list
class_names = common_classes

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png", "gif"])

if uploaded_file is not None:
    # Display the uploaded image
    image_data = uploaded_file.read()
    st.image(image_data, caption='Uploaded Image.', use_column_width=True)
    st.write("")
    st.write("Classifying...")

    # Preprocess the image
    img = Image.open(io.BytesIO(image_data))
    img = img.resize((IMG_WIDTH, IMG_HEIGHT))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    # Make prediction
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class_name = class_names[predicted_class_index]

    st.write(f"Prediction: **{predicted_class_name}**")

print("Streamlit application script 'app.py' created.")



# Instructions to run the Streamlit app
print("\nTo run the Streamlit app, save the above code as 'app.py' (which has been done) and then execute the following commands:")
print("!pip install pyngrok -q")
print("from pyngrok import ngrok")
print("public_url = ngrok.connect(8501)")
print("print(f\"Streamlit App URL: {public_url}\")")
print("!streamlit run app.py --server.port 8501 --server.enableCORS false --server.enableXsrfProtection false")


Writing app.py


In [33]:
import gradio as gr
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image
import io

print("Required libraries imported for Gradio application.")

Required libraries imported for Gradio application.


In [34]:
model = load_model('food_freshness_model.keras')

# Define class names from the common_classes list (already available in kernel state)
class_names = common_classes

def predict_freshness(img: Image.Image) -> str:
    # Preprocess the image
    img = img.resize((IMG_WIDTH, IMG_HEIGHT))
    img_array = np.asarray(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    # Make prediction
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class_name = class_names[predicted_class_index]

    return predicted_class_name

# Launch the Gradio interface
iface = gr.Interface(
    fn=predict_freshness,
    inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=gr.Label(),
    title="Food Freshness Classifier",
    description="Upload an image of food to classify its freshness (e.g., freshapples, rottenbanana)."
)

print("Gradio interface defined. To launch, run iface.launch().")


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Gradio interface defined. To launch, run iface.launch().


In [43]:
print("Launching Gradio interface. This might take a moment...")

# Launch the Gradio interface with share=True for public link in Colab
# The launch() method will print the public URL once it's ready.
iface.launch(share=True)

Launching Gradio interface. This might take a moment...
Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://82b12c3030dbe96ce6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


KeyboardInterrupt: 